# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/envs/aistudy6/lib/python3.11/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

# 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"
response, history = base_model.chat(tokenizer, query=input_text)
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中非常重要的一个卦象。乾卦代表着天、父亲、刚强、积极向上、行动力等。乾卦是由两个阴爻夹一个阳爻构成，象征着天父之威，以及阳刚之力量。

乾卦的含义非常丰富，它既象征着阳刚之力量，也象征着阳刚之精神。在易经中，乾卦通常被用来表示事物的成长、发展、积极向上、行动力等。同时，乾卦也象征着一种强烈的领导力、决断力、果敢力，以及对于事物的积极主动态度。

在八宫图说中，乾卦位于北方，与事业、努力、决断、领导、自信等有关。因此，乾卦也常常被用来指导人们在事业、工作、社交等方面应该具有的态度和行动方式。

总之，乾卦是易经中非常重要的一个卦象，它象征着阳刚之力量、精神、领导力等，对于人们的决策、行动等方面具有非常重要的指导意义。


### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [6]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它来自于易经。讼卦代表着诉讼、争端、诉讼纷争等。这个卦象是由两个阳爻夹一个阴爻构成，象征着阳刚之力量，但同时也暗示着诉讼和争端的可能性。

讼卦的含义非常丰富，它既象征着阳刚之力量，也象征着阳刚之精神。在易经中，讼卦通常被用来表示事物的争端、诉讼、纷争等。它告诉我们，在面对争端和诉讼时，应该保持冷静、客观、公正的态度，尽可能地避免争端和诉讼的发生。

在八宫图说中，讼卦位于西南方，与法律、公正、争端、诉讼等有关。因此，讼卦也常常被用来指导人们在法律、公正、争端、诉讼等方面应该具有的态度和行动方式。

总之，讼卦是易经中非常重要的一个卦象，它象征着阳刚之力量、精神、争端和诉讼等，对于人们的决策、行动等方面具有非常重要的指导意义。


# 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp

In [7]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250808_233529"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [8]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [9]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个阳爻（一一）和三个阴爻（oo）组成，形成一个卦象。这个卦象代表着天，预示着亨通，但也警告着小心谨慎。乾卦的核心哲学是：天行健，君子拟之。君子观天行健，从而学会行健，行止有序，速行。但需警惕，不可冒险行事。', 'content': '\n乾卦象征着天，预示着天行健。君子应观天行健，并模仿天行，行止有序，速行，而非冒险行事。同时，也预示着顺利、成功和权势。但在危险的情况下，需要小心谨慎行事。在事业上，应谨慎行事，计划行事，行动有序，取信于上，自我约束。在经商中，应谨慎经营，按商业规则行事，重信守诺，积极进取。在求名方面，应立志高远，奋发向上，积极进取，行事果敢，行事有序。在决策中，也应借鉴过去的经验，谨慎行事，自我约束，商业智慧与道德品质相结合。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_233529）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [10]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250808_233529）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [11]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '卜卦是由古代的占卜发展而来的一种预测方法，主要通过卜卦来预测事物的发展和结果。在占卜中，卜卦者会通过抛掷硬币或使用蓍草来得到一组卦象，然后根据卦象来询问卜者所卜之卦。通过对卦象的分析，可得到对事物的预测和解答。师卦是卜卦中的一种卦象，代表着军队的力量和实力，以及随之而来的吉祥。师卦的核心卦象是：兑为泽，下卦为坎，上卦为震。兑代表着喜悦和快乐，坎代表着云，震代表着雷。这里，震卦代表着君子观止自省，从而引发云雷之象。师卦的核心哲学是：云雷之象，君子观止自省，则罪过可赎，罪业消除。', 'content': '\n师卦的解释要点在于：\n\n1. 自我反省， corrected错误\n\n师卦的核心卦象，代表着占卜者所卜之卦，也是卜卦中的一种卦象。师卦代表着君子自我反省，引发云雷之象。这里，震卦代表着雷，象征着激发内省，而兑卦代表着喜悦，象征着引发内省的原因。\n\n师卦的核心哲学是：云雷之象，表明观卦者可以引发内省，从而 corrected错误。因此，师卦往往预示着占卜者可以消除罪恶，改正错误，重新开始，获得好运。\n\n2. 自我反省， corrected错误\n\n师卦的解释要点在于，占卜者需要自我反省，才能消除罪恶，改正错误。这里，君子观止自省，引发云雷之象。云雷之象，代表着激发内省，鼓励占卜者消除罪恶，改正错误。\n\n师卦的取卦区间：\n\n1. 事业经营：占卜者所占卜的卦象，与事业经营有关。 here，震卦代表着雷，象征着内省激发，使占卜者产生自我反省，从而引发云雷之象。\n\n2. 经商：占卜者所占卜的卦象，与经商有关。 here，兑卦代表着喜悦，代表着内省引发的好运。\n\n3. 求名：占卜者所占卜的卦象，与求名有关。 here，兑卦代表着喜悦，象征着内省引发的好运。\n\n4. 婚恋：占卜者所占卜的卦象，与婚恋有关。 here，坎卦代表着云，象征着内省引发的好运。\n\n5. 决策：占卜者所占卜的卦象，与决策有关。 here，震卦代表着雷，象征着激发内省。\n\n师卦的解释和好运相关，但需要自我反省才能消除罪恶，改正错误。因此，占卜者需要引发内省，消除罪恶，改正错误，从而获得好运。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250

# 其他模型（错误数据或训练参数）
### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [12]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250809_002946-error-dataset"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，屯卦是一个大吉大利的卦象，预示着吉祥和大利。然而，不利于出门，但有利于建国封侯。屯卦由上卦坎（水）下卦震（雷）组成，坎为云，震为雷。预示着云行雷动的卦象。君子观此卦象，取法于云雷，用云的恩泽，雷的威严来治理国事。屯卦象征着开始困难，需要毅力和果敢才能获得吉利。身处困境需要多加辛苦努力，排除困难，方可通达，有初难后解之象。因此，对于事业创业而言，应当小心翼翼，勇往直前，灵活机动，可望获得大的成功。但也需注意到仍有困难存在，务必有他人相助，平时应多施恩惠。对于经商，起初多有挫折，必须坚定信念，积极进取，行动果断，若仍无法摆脱困境，则应退守保全，等待机会，再展宏图。对于婚恋，好事多磨，忠贞纯洁，大胆追求，能够成功。屯卦的核心哲学在于，初难后解，需要毅力和坚忍不拔的毅力和锲而不舍的奋斗精神，但也需得到贤德之人的帮助才能摆脱困境。


In [14]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 蒙卦是由艮卦（山）下，坎卦（水）上组成的异卦相叠。它代表着通泰，启蒙的意义。在这里，卜者并非是在向幼稚愚昧的人取求，而是幼稚愚昧的人在向卜者求教。第一次卜筮就得到了神灵的指示。然而，如果轻慢不敬地再三卜筮的话，神灵便不会再示警。总的来说，这是一个吉利的卜问。

蒙卦的核心在于山下有泉的形象，寓意着启蒙。君子观此卦象，应当以果敢坚毅的行动来培养自身的品德，像山泉一样果断行动。然而，此卦乃是离宫四世卦，它代表着回还往复、疑惑不前、多忧愁过失，因而属于凶卦。

蒙卦在个人发展、事业经商、求名婚恋等方面的解释不一。在事业方面，表示事业初建，具有启蒙和通达之象，需要勇敢坚毅的行动；而在经商方面，需要务必小心谨慎，树立高尚的商业道德，不可急功近利；求名方面，需要接受良好的基础教育，陶冶情操。整体而言，此卦提示须忍耐待机而动，听取别人意见，方能通达运势。


In [15]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么概念 坤卦，是周易中的一卦，由两个坤卦叠加而成，代表大地的顺从和承载。在这个卦中，预示着大吉大利，预言了雌马牵动的吉兆，君子出行会先迷失，后来找到主人，有利的方向是西南，不利的方向是东北。总体上，这是一个吉利的卦象。《象辞》中讲到，大地形势平和，君子观卦以厚德载物。坤卦的解释中提到，坤卦代表柔顺和地气舒展之象，主张妥善安排，等待时机，宜顺从运势以制定大事。在传统解卦中，坤卦代表谨慎行事，灵活适应，依循正道获得吉利。在事业、经商、婚姻和决策等方面，坤卦均主张顺从自然规律，勿急进，谋求长远利益。


# 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [16]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-20250809_015451-lizhe"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [17]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中一个充满象征意义的卦象，由下卦坤（地）和上卦乾（天）组成。这一卦象生动地描绘了天地间的景象，象征着阳刚之气和宇宙运行的规律。乾卦的核心哲学启示我们，天气（乾）如同太阳运行在天空，必须遵循一定的运行轨迹，以确保地球上的生命能够正常生存。

在卦序发展上，乾卦紧随坤卦之后，这反映出宇宙中阴阳平衡的重要性。阴阳相摩罕有停歇，阴阳交合又必须掌握适度，这正是乾卦所要传达的"天行健"的哲学。

《序卦》中解释乾卦的顺序为：物极必反，阴极必阳，杂乱必乱，故有乾卦。这一理论说明在事物发展过程中，阴极（原创性）必然导致 Positive（阳）的转机，体现了宇宙运行的规律。

乾卦的卦义既蕴含了宇宙的运行规律，又体现了刚健准确的特定品质，是卦序发展的重要支撑。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 
乾卦是周易中一个充满力量和象征的卦象，由下卦坤（地）和上卦乾（天）组成。这一卦象生动地描绘了'天行健'的哲学内涵，即宇宙中天气运行的方式，象征着宇宙的运行力量和秩序。乾卦的核心象征意义是'键开键关'，这代表了掌握事物发展的关键能力。

乾卦包含着深刻的辩证关系：下卦坤代表柔顺，上卦乾代表刚健，两者结合意味着在柔顺中保持刚健，这正是推动事物发展的关键。卦辞明确指出'君子以恒其德'，强调只有坚守正道，才能保持持续的力量和成功。

在卦序演变中，乾卦紧随坤卦而来，这象征着宇宙中 '天行健' 的永恒规律。随着君子遵循君子之道，最终达到'亨通'的境界，这正是乾卦所代表的宇宙秩序和运行力量的最佳体现。


In [18]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
地水师卦是一个富有哲学意义的卦象，由下卦地（乾）和上卦水（兑）组成。这个卦象生动地展现了'干（乾）为天'的哲学理念，同时也'兑（兑）为泽'的卦象，因此这个卦象象征着地球（地）与水（泽）的相互关系。

在周易哲学中，天地之间的运行取决于阴阳两种基本力量。阴阳的相互作用使大地吸收了天气的力量，天气又影响着泽中的水。这种相互依存的关系正是这个卦象所要传达的深刻智慧。

地水师卦的核心思想是'天地交通'，这表示当阴阳两力相互融合时，天地之间的边界变得模糊。这种交通状态为人们提供了一个宝贵的机会：随着事物的发展，需要顺应这种新的状态。

从卦序来看，地水师卦紧随观卦（观卦的卦象是 thsky；enteq；n（天）和 zh（泽）组成），这表示在观察世界中，干天（乾）的性能开始体现。随着这种观察，兑泽（兑卦的卦象是 zh；eteq；n（泽）和 sh（ earth）组成）的特性开始显现，这表明在处理事物时，需要有天地交通的思维。

地水师卦启示我们：当遇到 conflicting forces 时，应该保持冷静，等待适当的时机使这两者融合。这不仅有利于实现天地交通的境界，还能为处理事物提供更好的策略。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 在周易哲学体系中，地水师卦是一个富有深邃智慧的卦象，由下卦地（乾）和上卦水（坤）组成。这一卦象生动地描绘了'地生水'的自然景象，象征着事物在发展过程中必然产生的先后顺序关系。在卦象构成上，乾卦代表天，坤卦代表地，二者结合形成天地的基础关系。

地水师卦的核心哲学启示我们：当事情发展进入先后序阶段时，必然会面临'先成后败'的规律。这不仅适用于自然界，也适用于人类社会。有时候，提前布局、耐心积累资源真的能带来更大的成功，而过于急躁的尝试反而可能导致失败的结果。

这一卦象特别适合那些需要耐心、持久策略的领域，如等待时机再行动，或者在团队建设、知识积累等方面都具有深刻的指导意义。周易智慧认为：事物的发展总是遵循一定的顺序规律，坚持正道最终将获得胜利。


In [19]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
天水讼卦是《周易》中一个充满矛盾与冲突的卦象，由乾（天）下乾（天）上组成，形成'天-天'的卦象。这一卦象生动地描绘了天空中的水（天水）与天空相争的景象，象征着现实世界中的矛盾与纠纷。卦象中，上乾下乾形成'干-天'的序列，生动地展现了天与地之间的冲突。

《周易》通过天水讼卦深刻地揭示了社会生活中矛盾冲突的难以避免性：如卦辞所说'利于诉讼'，表明在特定的社会背景下，冲突和纠纷是不可避免的，而这种状况对于君子来说既是挑战，也是机会。

从卦序来看，《序卦》指出'复则不妄加，故受之以讼'，说明在经历复卦（复卦序）之后，事物不再随意增减，这预示着讼卦（讼卦序）的出现。这表明在处理矛盾和纠纷时，需要遵循一定的规律，避免随意增减。

《象传》'天以下乾上，讼；君子以'观于水'”的解读，进一步深化了卦义，指出在处理天与地矛盾冲突的过程中，应当具备智慧与洞察力，如同观察水流的运行，从而体现公平义理。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 在周易哲学体系中，天水讼卦是一个富有辩证智慧的卦象，由下卦坤（地）和上卦乾（天）组成。这一卦象生动地描绘了'天在上，地在下'的宇宙关系，象征着人与自然的矛盾与冲突。卦象中心之处的卦辞'讼，指代了一种典型的宇宙现象：地球通过地壳运动产生地震，而地震又引发天体运动。这象征着宇宙中存在的一种自然辩证过程——由运动引发的运动。

天水讼卦的哲学启示在于：人生于这个世界，必然面临各种矛盾与纷争。就像宇宙中的两种力量相互交织，需要通过辩证过程达到和谐状态。这一卦象特别适用于面对重大决策时，提醒我们要谨慎思考：是否处于天时（阳）时势、是否处于人事（矛盾）时期。

卦辞中'天未助之，助之必吉'指出，当符合天道的决策时，积极行动将带来吉运；而违反卦义的行为则必受时机影响，此时静观等待时机才会带来吉果。

从卦序来看，《序卦》解释道：'物不可以终否，故受之以讼'，提示事物发展过程中，矛盾和纷争是不可避免的，必须面对和解决。而《象传》'君子以明庶政，先有曲直'指出，提醒执政者应明辨是非，保持公正，以达到避免纠纷的目的。
